In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

from scipy.spatial.distance import pdist, squareform

In [ ]:
data = pd.read_csv("tmdb_5000_movies.csv")
data = data.dropna(subset = ["overview"]) #vyhazuj jenom ty řádky, kde chybí overview
data.head()

X = data["overview"] # min_df =2  mělo by se to vyskytovat min. ve 2 popisech, když se vyskytují ve všech, tak nám nic neřeknou
# TFID Vectorizer - respektuje, jak je to slovo časté a jaká je jeho délka
vec = TfidfVectorizer(min_df=2, max_df=0.7, stop_words="english", ngram_range=(1, 2))
data_tfidf = vec.fit_transform(X)
# Převod na pole
#data_tfidf.toarray()

In [ ]:
data.shape

In [ ]:
data_tfidf = pd.DataFrame(data_tfidf.toarray(), columns=vec.get_feature_names_out(), index=data["title"])
data_tfidf.head()

In [ ]:
data_tfidf.loc["The Fugitive"].sort_values(ascending=False).head(20)

In [ ]:
# jak změříme, jestli je ten film podobný s jiným filmem

In [ ]:
fugitive = data_tfidf.loc["The Fugitive"].values.reshape(1,-1)  # z jednorozměrového pole udělá dvourozměrné
witness = data_tfidf.loc["Witness"].values.reshape(1,-1) 
titanic = data_tfidf.loc["Titanic"].values.reshape(1,-1) 

In [ ]:
print(cosine_similarity(fugitive,witness))
print(cosine_similarity(fugitive,titanic))

In [ ]:
cosine_similarity_array = cosine_similarity(data_tfidf)
# spočítá pro každou dvojici těch slov podobnost
# něco jako korelační matice
cosine_similarity_array

In [ ]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=data["title"], columns=data["title"])
cosine_similarity_df
cosine_similarity_df["The Fugitive"].sort_values(ascending=False).head(20)

In [ ]:
#Cvičení

data = pd.read_csv("tmdb_5000_movies.csv")
# Nastavíme index
data = data.set_index("title")
# Ponecháme v datech pouze sloupce od gen_Action po gen_Western
data_genres = data.loc[:, "gen_Action":"gen_Western"] # všechny řádky a všechy sloupce, které začínají na gen
data_genres.head()

In [ ]:
jaccard_distances = pdist(data_genres, metric = "jaccard")
jaccard_distances = squareform(jaccard_distances)
jaccard_distances = 1 - jaccard_distances

In [ ]:
data_genres.loc["The Fugitive"]

In [ ]:
jaccard_score = pd.DataFrame(jaccard_distances, index = data.index, columns = data.index)
jaccard_score

In [ ]:
jaccard_distances["The Fugitive"].sort_values()




In [ ]:
# Kolaborativní filtrování

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_score

In [ ]:
data_ratings = pd.read_csv("user_ratings.csv")
data_ratings = data_ratings[["userId", "rating", "title"]].drop_duplicates(subset=["userId", "title"], keep="last")  # berem jen to poslední hodnocení
data_ratings.head()
data_ratings.shape
data_ratings_pivot = data_ratings.pivot(index = "userId", columns = "title", values = "rating")

In [ ]:
data_ratings_pivot.loc[2].dropna().sort_values()

In [ ]:
data_ratings_pivot.loc[2].dropna().shape

In [ ]:
avg_ratings = data_ratings_pivot.mean() # uděláme průměrní hodnocení a odečteme to od dat
avg_ratings
# průměr je 4 (ale pro každý film je průměr jiný)
# někdo dal filmu hodnocení 3  ... 3-4 = -1 
# když odečtu od dat průměr
# hledáme uživatele, kteří mají podobný vkus
#5-4 = 1
#4-4 = 0
#3-4 = -1
#2-4 = -2
#1-4 = -3

In [ ]:
avg_ratings = data_ratings_pivot.mean()
data_ratings_pivot_standardized = data_ratings_pivot.sub(avg_ratings, axis=1)
data_ratings_pivot_standardized = data_ratings_pivot_standardized.fillna(0) #doplním nuly,tam, kde je nA
# pro ten daný řádek mi od těch hodnot odečte tento sloupec
data_ratings_pivot_standardized.head()


In [ ]:
similarities = cosine_similarity(data_ratings_pivot_standardized)
similarities = pd.DataFrame(similarities, index = data_ratings_pivot_standardized.index, columns = data_ratings_pivot_standardized.index)
similarities


In [ ]:
n_users = 10 #chceme se řídit podle 10-ti nejpodobnějších uživatelů #vzali jsme si 10 lidí 
user_2_similarities = similarities.loc[2] # vyberu řádek s uživatelem2
user_2_similarities # u 2 uživatele je 1 (to je sám se sebou)
user_2_similarities = user_2_similarities.sort_values(ascending=False).head(n_users)
user_2_similarities = user_2_similarities.iloc[1:]
user_2_similarities # to jsou idi, který jsou nejvíc podobné 2

In [ ]:
similar_users = user_2_similarities.index #vybrali jsme uživatele 2
similar_users # teď tam mám jenom ty uživatele (čísla těch uživatelů)

In [ ]:
# spočítali jsme si podobnost hodnocení všech dvojic uživatelů
# podívali jsme se, kteří uživatelé mají nejpodobnější hodnocení
data_ratings_pivot_similar = data_ratings_pivot.loc[similar_users] # budu tam mít ty původní hodnocení, ale uvažovat jen 10 lidí té skupiny similar users
data_ratings_pivot_similar = data_ratings_pivot_similar.dropna(axis=1, thresh = 3 ) # všechno jsou to filmy, které hodnotily z té skupiny alespoň 3 lidé
average_ratings_by_similar_users = data_ratings_pivot_similar.mean() # tady vidíme, jaké jsou průměrné hodnocení teď filmů
average_ratings_by_similar_users.sort_values(ascending=False) #nakonec jsme to seřadili, a 

In [ ]:
# mě teď budou zajímat filmy, které lidi z té skupiny 10 lidí viděli, a líbili se jim



In [ ]:
# Výběr na základě uživatelského profilu 

In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = pd.read_csv("tmdb_5000_movies.csv")
data = data.dropna(subset=["overview"])
X = data["overview"]
vec = TfidfVectorizer(min_df = 2, max_df = 0.7, stop_words="english",ngram_range=(1,2))
data_tfidf = vec.fit_transform(X)
data_tfidf = pd.DataFrame(data_tfidf.toarray(), columns = vec.get_feature_names_out(),index=data["title"])
cosine_similarity_array = cosine_similarity(data_tfidf)
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index = data_tfidf.index, columns = data_tfidf.index)

In [ ]:
favorite_movies = ["Notting Hill", "Titanic", "The Great Gatsby", "The Lovers"]
data_user_profile = data_tfidf.loc[favorite_movies] #
data_user_profile = data_user_profile.mean()
data_user_profile = data_user_profile.values.reshape(1, -1)

data_tfidf_not_seen = data_tfidf.drop(favorite_movies, axis=0)
cosine_similarity_not_seen = cosine_similarity(data_tfidf_not_seen, data_user_profile)
cosine_similarity_not_seen = pd.DataFrame(cosine_similarity_not_seen, index=data_tfidf_not_seen.index,columns=["score"])

cosine_similarity_not_seen.sort_values(by="score", ascending=False).head(20)


In [ ]:
# cvičení

In [ ]:
data = pd.read_csv("netflix_titles.csv")
data.head()

In [15]:
data = pd.read_csv("netflix_titles.csv")
data = data.dropna(subset=["description"])
X = data["description"]
vec = TfidfVectorizer(min_df=2, max_df=0.7, stop_words="english", ngram_range=(1,2,))
data_tfidf = vec.fit_transform(X)
#vec.get_feature_names_out()
cosine_similarity_array = cosine_similarity(data_tfidf)
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=data["title"], columns=data["title"])
cosine_similarity_df["La casa de papel"].sort_values(ascending=False)

title
La casa de papel                     1.000000
V.R. Troopers                        0.232403
Power Rangers Time Force             0.228176
Don 2                                0.222125
Inside Man: Most Wanted              0.185587
                                       ...   
The Ranch                            0.000000
The goop lab with Gwyneth Paltrow    0.000000
Sons of the Caliphate                0.000000
Rise of Empires: Ottoman             0.000000
Zubaan                               0.000000
Name: La casa de papel, Length: 8807, dtype: float64